In [ ]:
!pip3 install -qU langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 827.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [ ]:
import google.generativeai as genai
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
!pip3 install cdlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.8 MB/s eta 0:00:00


In [ ]:
!pip3 install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.0 MB/s eta 0:00:00


In [ ]:
!pip3 install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import networkx as nx
from cdlib import algorithms
import os
import shutil
from langchain import PromptTemplate
import time
from langchain_google_genai import ChatGoogleGenerativeAI

Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'infomap', 'graph_tool', 'wurlitzer', 'leidenalg'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'wurlitzer', 'infomap'}


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
import time
class counter():
    def __init__(self):
        self.count = 0

    def increment(self):
        self.count += 1
        if (self.count >= 15):
            self.count = 1
            time.sleep(60)

In [ ]:
calls_track = counter()

In [ ]:
# 1. Source Documents → Text Chunks
def split_documents_into_chunks(documents, chunk_size=600, overlap_size=100):
    chunks = []
    for document in documents:
        for i in range(0, len(document), chunk_size - overlap_size):
            chunk = document[i: i+chunk_size]
            chunks.append(chunk)
    return chunks

In [ ]:
# 2. Text Chunks → Element Instances
def extract_elements_from_chunks(chunks):
    elements = []
    for index, chunk in enumerate(chunks):
        print(f"Chunk index {index} of {len(chunks)}:")
        calls_track.increment()
        messages = [
            (
                "system",
                "Extract entities and relationships from the following text.",
            ),
            ("human", chunk),
        ]

        ai_msg = llm.invoke(messages)
        print(ai_msg.content)
        entities_and_relations = ai_msg.content
        elements.append(entities_and_relations)

    return elements

In [ ]:
# 3. Element Instances → Element Summaries
def summarize_elements(elements):
    summaries = []
    count = 0
    for index, element in enumerate(elements):
        print(f"Element index {index} of {len(elements)}:")
        calls_track.increment()
        messages = [
            (
                "system",
                "Summarize the following entities and relationships in a structured format. Use \"->\" to represent relationships, after the \"Relationships:\" word",
            ),
            ("human", element),
        ]

        ai_msg = llm.invoke(messages)
        print(ai_msg.content)
        summary = ai_msg.content
        print("Element Summary:", summary)
        summaries.append(summary)
    return summaries

In [ ]:
# 4. Element Summaries → Graph Communities
def build_graph_from_summaries(summaries):
    G = nx.Graph()
    for index, summary in enumerate(summaries):
        print(f"Summary index {index} of {len(summaries)}:")
        lines = summary.split("\n")
        entities_section = False
        relationships_section = False
        entities = []
        for line in lines:
            if line.startswith("### Entities:") or line.startswith("**Entities:**"):
                entities_section = True
                relationships_section = False
                continue
            elif line.startswith("### Relationships:") or line.startswith("**Relationships:**"):
                entities_section = False
                relationships_section = True
                continue
            if entities_section and line.strip():
                if line[0].isdigit() and line[1] == ".":
                    line = line.split(".", 1)[1].strip()
                entity = line.strip()
                entity = entity.replace("**", "")
                entities.append(entity)
                G.add_node(entity)
            elif relationships_section and line.strip():
                parts = line.split("->")
                if len(parts) >= 2:
                    source = parts[0].strip()
                    target = parts[-1].strip()
                    relation = " -> ".join(parts[1:-1]).strip()
                    G.add_edge(source, target, label=relation)
    return G

In [ ]:
# 5. Graph Communities → Community Summaries
def detect_communities(graph):
    communities = []
    index = 0
    for component in nx.connected_components(graph):
        print(
            f"Component index {index} of {len(list(nx.connected_components(graph)))}:")
        subgraph = graph.subgraph(component)
        if len(subgraph.nodes) > 1:  # Leiden algorithm requires at least 2 nodes
            try:
                sub_communities = algorithms.leiden(subgraph)
                for community in sub_communities.communities:
                    communities.append(list(community))
            except Exception as e:
                print(f"Error processing community {index}: {e}")
        else:
            communities.append(list(subgraph.nodes))
        index += 1
    print("Communities from detect_communities:", communities)
    return communities


def summarize_communities(communities, graph):
    community_summaries = []
    count = 0
    for index, community in enumerate(communities):
        print(f"Summarize Community index {index} of {len(communities)}:")
        subgraph = graph.subgraph(community)
        # nodes = list(subgraph.nodes)
        # edges = list(subgraph.edges(data=True))

        # Rank nodes by degree (to prioritize prominent nodes)
        sorted_nodes = sorted(subgraph.nodes, key=lambda n: subgraph.degree(n), reverse=True)
        # Rank edges by combined node degrees (source degree + target degree)
        sorted_edges = sorted(subgraph.edges(data=True), key=lambda e: subgraph.degree(e[0]) + subgraph.degree(e[1]), reverse=True)

        nodes = list(sorted_nodes)
        edges = list(sorted_edges)

        description = "Entities: " + ", ".join(nodes) + "\nRelationships: "
        relationships = []
        for edge in edges:
            relationships.append(
                f"{edge[0]} -> {edge[2]['label']} -> {edge[1]}")
        description += ", ".join(relationships)
        calls_track.increment()
        messages = [
            (
                "system",
                "Summarize the following community of entities and relationships.",
            ),
            ("human", description),
        ]

        ai_msg = llm.invoke(messages)
        print(ai_msg.content)
        summary = ai_msg.content.strip()
        print("Community Summary: ", summary)
        community_summaries.append(summary)

    return community_summaries

In [ ]:
# 6. Community Summaries → Community Answers → Global Answer
def generate_answers_from_communities(community_summaries, query):
    intermediate_answers = []
    count = 0
    for index, summary in enumerate(community_summaries):
        print(f"Summary index {index} of {len(community_summaries)}:")
        calls_track.increment()
        messages = [
            (
                "system",
                "Answer the following query based on the provided summary and also generate a score between 0 to 100 as to how helpful the generated answer is to answer the query in the format #$%Score->score generated#$%.",
            ),
            ("human", f"Query: {query} Summary: {summary}"),
        ]

        ai_msg = llm.invoke(messages)
        print(ai_msg.content)

        l1 = ai_msg.content.split('#$%')
        score = 0
        for i in range(len(l1)):
            if (l1[i: i+5] == 'Score'):
                score = int(l1[7: ])
                break

        result = ai_msg.content.strip()
        print("Intermediate answer:", result)
        intermediate_answers.append([score, result])
    calls_track.increment()
    sorted_lst = sorted(intermediate_answers, key=lambda x: x[0])

    print(sorted_lst)
    answers = []
    for i in range(len(sorted_lst)):
        if (sorted_lst[i][0] < 20):
            continue
        answers.append(sorted_lst[i][1])
    messages = [
        (
            "system",
            "Combine these answers into a final, concise response.",
        ),
        ("human", f"Intermediate answers: {answers}"),
    ]

    ai_msg = llm.invoke(messages)
    print(ai_msg.content)
    result = ai_msg.content.strip()
    print("Final answer:", result)
    final_answer = result
    return final_answer

In [ ]:
# Putting It All Together
def graph_rag_pipeline(documents, query, chunk_size=600, overlap_size=100):
    # Step 1: Split documents into chunks
    chunks = split_documents_into_chunks(documents, chunk_size, overlap_size)

    # Step 2: Extract elements from chunks
    elements = extract_elements_from_chunks(chunks)
    with open('elements.txt', 'w+') as f:
        for i in range(len(elements)):
            f.write(elements[i])

    # Step 3: Summarize elements
    summaries = summarize_elements(elements)
    with open('summaries.txt', 'w+') as f:
        for i in range(len(summaries)):
            f.write(summaries[i])

    # Step 4: Build graph and detect communities
    graph = build_graph_from_summaries(summaries)
    print("graph:", graph)
    communities = detect_communities(graph)

    print("communities:", communities[0])
    # Step 5: Summarize communities
    community_summaries = summarize_communities(communities, graph)
    with open('community_summaries.txt', 'w+') as f:
        for i in range(len(community_summaries)):
            f.write(community_summaries[i])

    # Step 6: Generate answers from community summaries
    final_answer = generate_answers_from_communities(
        community_summaries, query)

    return final_answer, elements, summaries, community_summaries

In [ ]:
import json

with open('/content/Save_dataset_documents.json', 'r') as f:
    d = json.load(f)
    print(d)

{'Document 1': '                                                                                      UNITED STATES\n                                                          SECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549\n\n                                                                                       FORM 10-Q\n(Mark One)\n                          ☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n                                                                         For the quarterly period ended June 25, 2022\n                                                                                                    or\n                          ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n                                                                       For the transition period from                  to           .\n                                                              

In [ ]:
DOCUMENTS = []
for i in d:
    DOCUMENTS.append(d[i])

In [ ]:
query = "What are the main themes in these documents?"
print('Query:', query)
answer, elements, summaries, community_summaries = graph_rag_pipeline(DOCUMENTS, query)
print('Answer:', answer)

Streaming output truncated to the last 5000 lines.
Once I have this information, I can identify the main themes related to the address.
Summary index 80 of 422:
The main theme in the provided summary is the **relationship between the IRS Employer Identification Number (EIN) and other entities**.  The summary specifically requests information about how the EIN "94-1672743" is connected to other entities, such as companies, individuals, and tax filings. 

To effectively summarize the community of entities, you need to provide the specific relationships between the EIN and other entities.  Without this information, it's impossible to understand the connections and interactions within the community. 

Intermediate answer: The main theme in the provided summary is the **relationship between the IRS Employer Identification Number (EIN) and other entities**.  The summary specifically requests information about how the EIN "94-1672743" is connected to other entities, such as companies, individ

In [ ]:
with open('./summaries.txt', 'r') as f:
    x = f.read()
x

'## Entities and Relationships:\n\n**Entities:**\n\n* **SEC (United States Securities and Exchange Commission)**\n* **Company**\n\n**Relationships:**\n\n* **Company -> Form 10-Q -> SEC**\n* **Form 10-Q -> Quarterly Report**\n* **Quarterly Report -> Section 13 or 15(d) of the Securities Exchange Act of 1934**\n* **Quarterly Report -> June 25, 2022 (quarterly period end date)**\n* **Company -> Securities Exchange Act of 1934** \n\n**Additional Information:**\n\n* **SEC -> Washington, D.C. 20549 (location)** \n## Entities and Relationships Summary:\n\n**Entities:**\n\n* Company\n* SEC (Securities and Exchange Commission)\n* Section 13/15(d) (Sections of the Securities Exchange Act of 1934)\n* Transition Report (unclear nature)\n\n**Relationships:**\n\n* Company -> Filing -> SEC\n* Filing -> Regulation -> Section 13/15(d)\n* Filing -> Time Period -> June 25, 2022 (quarterly period ended)\n* Company -> File Number -> 001-36743\n* Company -> Transition Report (unclear relationship) \n## Enti

In [ ]:
index = 0
summaries = []
while index < len(x):
    i = index+2
    string = ''
    while (i+1 < len(x) and not (x[i] == '#' and x[i+1] == '#')):
        i += 1
    string = x[index: i]
    index = i
    summaries.append(string)

In [ ]:
query = "What were the risk factors identified by Intel in its Q3 2023 10-Q, especially any that are newly emerging or increasing?"     # single doc single chunk query case

# Step 4: Build graph and detect communities
graph = build_graph_from_summaries(summaries)
print("graph:", graph)
communities = detect_communities(graph)

print("communities:", communities[0])
# Step 5: Summarize communities
community_summaries = summarize_communities(communities, graph)
with open('community_summaries_updated.txt', 'w+') as f:
    for i in range(len(community_summaries)):
        f.write(f'##{i = }: ')
        f.write(community_summaries[i])
        f.write('\n\n')

# Step 6: Generate answers from community summaries
final_answer = generate_answers_from_communities(
    community_summaries, query)

Streaming output truncated to the last 5000 lines.
For example, tell me:

* **What other entities are involved?**  (e.g., companies, investors, government agencies, etc.)
* **What are the types of relationships?** (e.g., regulates, governs, requires, protects, etc.)

Once you provide this information, I can give you a concise summary.
Summarize Community index 360 of 422:
Please provide the relationships for the entity "Commission file number 0-23985".  Without the relationships, I cannot summarize the community of entities. 

For example, you could provide relationships like:

* **Is assigned to:** Company X
* **Is used in:** Filing Y
* **Is referenced in:** Document Z

Once you provide the relationships, I can give you a concise summary of the community of entities. 

Community Summary:  Please provide the relationships for the entity "Commission file number 0-23985".  Without the relationships, I cannot summarize the community of entities. 

For example, you could provide relationsh

In [ ]:
with open('./community_summaries.txt', 'r') as f:
    community_summaries1 = f.read()
community_summaries1

In [ ]:
query = "What were the risk factors identified by Intel in its Q3 2023 10-Q, especially any that are newly emerging or increasing?"
final_answer = generate_answers_from_communities(community_summaries1, query)

Summary index 0 of 423:
I cannot answer your query. The provided summary does not contain any information about Intel's Q3 2023 10-Q or any risk factors identified within it. 

#$%Score->0#$% 

Intermediate answer: I cannot answer your query. The provided summary does not contain any information about Intel's Q3 2023 10-Q or any risk factors identified within it. 

#$%Score->0#$%
Summary index 1 of 423:
The provided summary does not contain information about Intel's Q3 2023 10-Q or any risk factors identified within it. Therefore, I cannot answer your query. 

#$%Score->0#$% 

Intermediate answer: The provided summary does not contain information about Intel's Q3 2023 10-Q or any risk factors identified within it. Therefore, I cannot answer your query. 

#$%Score->0#$%
Summary index 2 of 423:
The provided summary does not contain information about Intel's Q3 2023 10-Q or any risk factors identified in it. Therefore, I cannot answer your query. 

#$%Score->0#$% 

Intermediate answer: Th

KeyboardInterrupt: 